<a href="https://colab.research.google.com/github/ashsingh138/Hackathon-1-summer-Analytics/blob/main/hackathon_1_Summer_Analytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded=files.upload()

Saving hacktest.csv to hacktest.csv


In [ ]:
from google.colab import files
uploaded=files.upload()

Saving hacktrain.csv to hacktrain.csv


In [ ]:
pip install catboost


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 8.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import RidgeClassifier
from sklearn.metrics import accuracy_score
import lightgbm as lgb
import catboost as cb
import torch
import torch.nn as nn


train = pd.read_csv('hacktrain.csv')
test = pd.read_csv('hacktest.csv')
ndvi_cols = [col for col in train.columns if '_N' in col]
imputer = KNNImputer(n_neighbors=3)
train[ndvi_cols] = imputer.fit_transform(train[ndvi_cols])
test[ndvi_cols] = imputer.transform(test[ndvi_cols])
le = LabelEncoder()
train['class_enc'] = le.fit_transform(train['class'])


def extract_features(df):
    X = df[ndvi_cols].values
    feats = np.c_[
        X.mean(axis=1), X.std(axis=1), X.min(axis=1), X.max(axis=1),
        np.median(X, axis=1), np.percentile(X, 25, axis=1),
        np.percentile(X, 75, axis=1), X[:, -1] - X[:, 0]
    ]
    return np.hstack([X, feats])

X_train = extract_features(train)
X_test = extract_features(test)
y_train = train['class_enc'].values


class FCNN(nn.Module):
    def __init__(self, in_dim, n_classes):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, 128), nn.ReLU(), nn.BatchNorm1d(128), nn.Dropout(0.3),
            nn.Linear(128, 64), nn.ReLU(), nn.BatchNorm1d(64), nn.Dropout(0.2),
            nn.Linear(64, n_classes)
        )
    def forward(self, x): return self.net(x)

def train_fcnn(X, y, X_test, n_classes, n_folds=5):
    skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)
    oof = np.zeros((len(X), n_classes))
    preds = np.zeros((len(X_test), n_classes))
    for train_idx, val_idx in skf.split(X, y):
        model = FCNN(X.shape[1], n_classes)
        optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
        criterion = nn.CrossEntropyLoss()
        X_tr = torch.tensor(X[train_idx]).float()
        y_tr = torch.tensor(y[train_idx]).long()
        X_val = torch.tensor(X[val_idx]).float()
        y_val = torch.tensor(y[val_idx]).long()
        X_te = torch.tensor(X_test).float()
        for epoch in range(15):
            model.train()
            optimizer.zero_grad()
            logits = model(X_tr)
            loss = criterion(logits, y_tr)
            loss.backward()
            optimizer.step()
        model.eval()
        oof[val_idx] = model(X_val).softmax(1).detach().numpy()
        preds += model(X_te).softmax(1).detach().numpy() / n_folds
    return oof, preds


fcnn_oof, fcnn_preds = train_fcnn(X_train, y_train, X_test, len(le.classes_))


lgbm = lgb.LGBMClassifier(n_estimators=500)
catb = cb.CatBoostClassifier(iterations=500, verbose=0)
lgbm.fit(X_train, y_train)
catb.fit(X_train, y_train)
lgbm_oof = lgbm.predict_proba(X_train)
catb_oof = catb.predict_proba(X_train)
lgbm_preds = lgbm.predict_proba(X_test)
catb_preds = catb.predict_proba(X_test)

stack_train = np.hstack([fcnn_oof, lgbm_oof, catb_oof])
stack_test = np.hstack([fcnn_preds, lgbm_preds, catb_preds])
meta = RidgeClassifier()
meta.fit(stack_train, y_train)
final_preds = meta.predict(stack_test)


submission = pd.DataFrame({
    'ID': test['ID'],
    'class': le.inverse_transform(final_preds)
})
submission.to_csv('submission.csv', index=False)
print("Submission saved as submission.csv")


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004198 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 8925
[LightGBM] [Info] Number of data points in the train set: 8000, number of used features: 35
[LightGBM] [Info] Start training from score -2.252605
[LightGBM] [Info] Start training from score -0.261527
[LightGBM] [Info] Start training from score -3.709082
[LightGBM] [Info] Start training from score -2.481413
[LightGBM] [Info] Start training from score -5.585999
[LightGBM] [Info] Start training from score -4.333236
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [W

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


Submission saved as submission.csv
